# RAGBOT

#### SIMPLE RAG CHATBOT 

chatbot that allows users to chat with custom data sources

In [ ]:
## Uploading the pdf file
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("universe.pdf")
docs=loader.load()
docs

[Document(metadata={'producer': 'pdfTeX-1.40.17', 'creator': 'LaTeX with hyperref package', 'creationdate': '2020-01-28T01:42:35+00:00', 'author': '', 'keywords': '', 'moddate': '2020-01-28T01:42:35+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.17 (TeX Live 2016) kpathsea version 6.2.2', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'universe.pdf', 'total_pages': 16, 'page': 0, 'page_label': '1'}, page_content='The Formation of the Universe: Conjectures\nMaurice Kleman\nJanuary 28, 2020\nInstitut de Physique du Globe de Paris\nUniversit´ e Sorbonne Paris Cit´ e\n1 rue Jussieu 75238 Paris cedex 05 France\nAbstract\nThis article puts forward a model of the formation of the universe,\nwhose essential novel ingredient is a pre-Universe reservoir RU with nei-\nther space nor time dimensions, in interaction with the universe U. U\nresults from a process of apparition of spacetime entitiesemanating from\nthe ur-elements in R U. The analysis of this appar

In [ ]:
#We are spliiting text content for the llm to intake.into chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000,chunk_overlap=400)
doc1 = text_splitter.split_documents(docs)
doc1

[Document(metadata={'producer': 'pdfTeX-1.40.17', 'creator': 'LaTeX with hyperref package', 'creationdate': '2020-01-28T01:42:35+00:00', 'author': '', 'keywords': '', 'moddate': '2020-01-28T01:42:35+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.17 (TeX Live 2016) kpathsea version 6.2.2', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'universe.pdf', 'total_pages': 16, 'page': 0, 'page_label': '1'}, page_content='The Formation of the Universe: Conjectures\nMaurice Kleman\nJanuary 28, 2020\nInstitut de Physique du Globe de Paris\nUniversit´ e Sorbonne Paris Cit´ e\n1 rue Jussieu 75238 Paris cedex 05 France\nAbstract\nThis article puts forward a model of the formation of the universe,\nwhose essential novel ingredient is a pre-Universe reservoir RU with nei-\nther space nor time dimensions, in interaction with the universe U. U\nresults from a process of apparition of spacetime entitiesemanating from\nthe ur-elements in R U. The analysis of this appar

In [ ]:
#Setting environment 
import os
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
#Setting API key 
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")

In [ ]:
#importing lib for embedding and vector stores
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings=GoogleGenerativeAIEmbeddings(model="models/embedding-001")

embeddings

GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000002382F178590>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x000002382F179BE0>, model='models/embedding-001', task_type=None, google_api_key=SecretStr('**********'), credentials=None, client_options=None, transport=None, request_options=None)

In [6]:
db=Chroma.from_documents(doc1,GoogleGenerativeAIEmbeddings(model="gemini-embedding-001"))

In [7]:
db

In [ ]:
#asking query and data is fetched from db by similarity search
query = "The variation of the internal energy of the Universe with time"
result=db.similarity_search(query)
result[0].page_content

'This equation of state is well-established in cosmology and is considered at the\norigin of the inﬂation of the universe [19] at its very early beginning. It is strictly\nvalid at the ﬁrst appearing spacetime entity, when V = 0.\nThe foregoing considerations do not provide the R U →U Zermelo process\nwith special features and are not very appealing, except for the existence of\na negative pressure. Generally, the thermodynamic equilibrium between two\nsystems in contact implies the equality of the temperatures Ta = Tb and of the\nchemical potentials µa = µb. The assumption that R U and U are in equilibrium\nis a notion diﬃcult to assess insofar as there is no geometrical contact (but in\na sense R U and U are in contact everywhere where U exists) and furthermore\nthere is no deﬁned temperature in R U. As stated above, the chemical potential\nand the temperature of a relativistic system in thermodynamic equilibrium are\nnot constrained to be spatially constant; but the ratioµ/T has to 

In [9]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002382F830AD0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002382F8316A0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [10]:
#chatprompt template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based in the context provided only.
Think step by step before answering the question and provide detailed answer.
<context>
{context}                                      
</context>
Question : {input}                                   
""")

In [11]:
from langchain.chains.combine_documents import create_stuff_documents_chain
#Takes a list of documents, combines (“stuffs”) them into a single prompt
doc_chain = create_stuff_documents_chain(model,prompt)

In [12]:
doc_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nAnswer the following question based in the context provided only.\nThink step by step before answering the question and provide detailed answer.\n<context>\n{context}                                      \n</context>\nQuestion : {input}                                   \n'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002382F830AD0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002382F8316A0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))
| St

In [13]:
# retriever 
# it is a interface where it returns/retrieves docs from a query given. It fetches

In [14]:
retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000002382F17AF90>, search_kwargs={})

In [15]:
#now we are ginna combine retriver and doc_chain ...then it becomes retirever chain!!

In [16]:
# create the retriver chain
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,doc_chain)

#### ASK QUESTIONS

In [22]:
response= retrieval_chain.invoke({"input":"give a summary of this "})

In [23]:
response["answer"]

'This paper proposes a model for the formation of the universe, built on the idea that there exists a pre-universe reservoir, denoted as RU, lacking space and time dimensions. \n\nHere are the key points:\n\n* **RU and the Universe (U):** The universe (U) emerges from RU through the "apparition" of spacetime entities. \n* **Thermodynamics:** The model applies the second law of thermodynamics and the principle of conservation of energy to the closed system U ∪ RU.  \n* **Entropy Transfer:** The second law dictates that entropy flows from RU to U. Each spacetime entity contributes an entropy of kB ln 2.\n* **Spacetime Entities:** These entities are interpreted as elementary black holes (EBHs) at the Planck scale. They possess various properties like angular momentum and electric charge, and their distribution follows a Boltzmann distribution.\n* **Temperature of Apparition:**  Assuming the total mass-energy of EBHs equals the observable universe\'s mass-energy, the model calculates a tem